## 测试数据库连接

In [1]:
from pymysql import Connection

# Initialize database connection and get schema description
def init_db(host:str, port:str, user:str, password:str, database:str) -> Connection:
    return Connection(host=host, port=int(port), user=user, password=password, database=database)

def get_db_schema(conn:Connection, db_name:str):
    sql = f"show tables from {db_name}"
    with conn.cursor() as cursor:
        cursor.execute(sql)
        tables = cursor.fetchall()
        tables_schema = []
        for table in tables:
            sql = f"show create table {table[0]}"
            cursor.execute(sql)
            create_table_sql = cursor.fetchone()
            tables_schema.append(create_table_sql)
        return tables_schema

from config import *

dbConnection = init_db(HOST, PORT, USER, PASSWORD, DATABASENAME)
schema = get_db_schema(dbConnection, DATABASENAME)
print(schema)


[('ACT_EVT_LOG', "CREATE TABLE `ACT_EVT_LOG` (\n  `LOG_NR_` bigint NOT NULL AUTO_INCREMENT,\n  `TYPE_` varchar(64) CHARACTER SET utf8mb3 COLLATE utf8mb3_bin DEFAULT NULL,\n  `PROC_DEF_ID_` varchar(64) CHARACTER SET utf8mb3 COLLATE utf8mb3_bin DEFAULT NULL,\n  `PROC_INST_ID_` varchar(64) CHARACTER SET utf8mb3 COLLATE utf8mb3_bin DEFAULT NULL,\n  `EXECUTION_ID_` varchar(64) CHARACTER SET utf8mb3 COLLATE utf8mb3_bin DEFAULT NULL,\n  `TASK_ID_` varchar(64) CHARACTER SET utf8mb3 COLLATE utf8mb3_bin DEFAULT NULL,\n  `TIME_STAMP_` timestamp(3) NOT NULL DEFAULT CURRENT_TIMESTAMP(3),\n  `USER_ID_` varchar(255) CHARACTER SET utf8mb3 COLLATE utf8mb3_bin DEFAULT NULL,\n  `DATA_` longblob,\n  `LOCK_OWNER_` varchar(255) CHARACTER SET utf8mb3 COLLATE utf8mb3_bin DEFAULT NULL,\n  `LOCK_TIME_` timestamp(3) NULL DEFAULT NULL,\n  `IS_PROCESSED_` tinyint DEFAULT '0',\n  PRIMARY KEY (`LOG_NR_`)\n) ENGINE=InnoDB DEFAULT CHARSET=utf8mb3 COLLATE=utf8mb3_bin"), ('ACT_GE_BYTEARRAY', 'CREATE TABLE `ACT_GE_BYTEAR

## 测试agent路径

In [ ]:
question = 'ABCDE菜餐饮管理有限公司的联系电话和地址分别是？'
question = '你好呀！'

import json
from openai import OpenAI
from config import *
from tool import *

client = OpenAI(
    base_url=BASE_URL_BAILIAN,
    api_key=DASHSCOPE_API_KEY,
)

system_prompt = f"""
你是一位专业的DBA，根据以下<schema>中的表结构，根据用户的问题编写SQL语句，允许使用模糊匹配 LINK操作符。
<schema> {schema} </schema>
"""

response = client.chat.completions.create(
    model="qwen2.5-14b-instruct",
    messages=[{"role":"system","content":system_prompt},{"role":"user","content":question}],
    tools=TOOLS,
    tool_choice="auto",
)

print(response.choices[0].finish_reason)

if response.choices[0].finish_reason == "tool_calls":
    for tool_call in response.choices[0].message.tool_calls:
        function_name = tool_call.function.name
        function_args = json.loads(tool_call.function.arguments)

        if function_name == "search_mysql":
            answer = search_mysql(schema,dbConnection,function_args["question"])
            print(answer)
            
# print(response)


## prompt测试

In [ ]:
# 当进行大模型的二次询问时，目的是为了更准确地理解用户的意图和需求，确保提供最相关和有用的信息。这通常发生在初次询问不够具体或清晰，或者初次回应后用户提供了额外信息的情况下。

# 如果你是开发者或操作者，想要设计一个有效的二次询问策略，可以考虑以下几个方面：

# 澄清问题：如果用户的初次询问模糊不清，可以通过提问来获取更多细节。例如，“您提到的‘这个’具体指的是什么呢？”或“能否提供更多关于您的需求的背景信息？”
# 确认理解：在给出答案之前，先总结你对用户需求的理解，并请求确认。“我的理解是您需要...，对吗？” 这样可以确保提供的信息是用户真正需要的。
# 细化选项：如果用户的需求可能对应多个解决方案，提出细化的选择可以帮助他们更好地表达自己的偏好。“您是想了解A方面，还是B方面的内容呢？”
# 引导性问题：通过提出一些引导性的问题，可以帮助用户思考并明确他们的实际需求。“您是在寻找一种新的方法来解决这个问题，还是希望改进现有的方案？”
# 反馈机制：建立一个简单的反馈机制，让用户能够指出回答中的不足之处，从而进一步调整和优化回答。“以上信息是否解决了您的问题？如果没有，请告诉我还需要哪些帮助。”

In [7]:
# 读取txt文件中的所有内容
def read_txt(file_path):
    with open(file_path, 'r') as f:
        content = f.read()
    return content
schema = read_txt("./schema_output.txt")

In [14]:

system_prompt = f"""

    你是一个数据库查询专家，你需要根据用户问题生成数据库查询语句，用户并不了解数据库相关知识，请不要尝试询问用户数据库的相关知识
    
    确保生成的查询语句的准确和完整。

    数据库表结构如下：
    {schema}

    如果根据表结构信息，和用户提问的内容无法确定sql语句，请尝试向用户提问，让用户重新表达问题。

    若可以确定sql语句，则直接返回一条pymysql可执行的语句，不要添加其他任何的符号或说明。

    要求尽可能的提高查询效率

    注意你只能返回一条sql语句，若当前问题无法依靠一条sql语句完成，你尝试让用户尽可能完善表述然后重新提问
    
"""
from openai import OpenAI 
from config import *

client = OpenAI(
    base_url=BASE_URL_BAILIAN,
    api_key=DASHSCOPE_API_KEY,
)


question = """
    查询所有信息
"""

response = client.chat.completions.create(
    model="qwen2.5-32b-instruct",
    messages=[{"role":"system","content":system_prompt},{"role":"user","content":question}],
    max_tokens=96,
)
print(response.choices[0].message.content)

请明确您想要查询哪个表或哪些表中的所有信息，或者是特定字段的信息。这样我可以帮助您构建更精确的SQL查询语句。例如，如果您想查询`assets_accounting_main`表中的所有信息，您可以告诉我。


In [16]:

system_prompt = f"""

   你是一个需求确认专家，你需要根据用户需求，不断询问用户相关问题，以完善需求，直到获得一个完整的需求。
    
"""
from openai import OpenAI 
from config import *

client = OpenAI(
    base_url=BASE_URL_BAILIAN,
    api_key=DASHSCOPE_API_KEY,
)


question = """
    查询有多少员工
"""

response = client.chat.completions.create(
    model="qwen2.5-14b-instruct",
    messages=[{"role":"system","content":system_prompt},{"role":"user","content":question}],
    max_tokens=96,
)
print(response.choices[0].message.content)

为了更好地理解您的需求，请回答以下几个问题：

1. 您希望查询的是整个公司的所有员工数量吗？还是特定部门的员工数量？
2. 是否需要按照某个特定条件（例如入职时间、职位等）来筛选员工？
3. 这个查询是用于日常报告还是偶尔需要的统计信息？
4. 是否需要同时获取其他相关信息，如员工总数中的全职与兼职比例？
5. 是否有特定的时间范围限制，


In [ ]:
# 再写一个tool 进行需求判定？

# 使用集成学习的方法，使用高温得到不同的sql语句，然后选择返回结果最多的结果，作为模型输出